# 1. Import Data

In [1]:
# Standard libs
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Tuple, List
import os
import glob
import json

# Utils
import math
import random
import gc

# Data & Processing
import numpy as np
import pandas as pd
import geopandas as gpd

# Scikit-Learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, confusion_matrix
from sklearn.neighbors import BallTree, NearestNeighbors
from sklearn.model_selection import train_test_split

# TQDM
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

# Plotting
import matplotlib.pyplot as plt

# PyTorch
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

# PyTorch Geometric
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GatedGraphConv, GATConv


# Confirm the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /home/qusta100/STGNN


In [7]:
#Load data
df = pd.read_csv(
    "/gpfs/scratch/qusta100/STGNN/Data/Temp/final.csv",
    dtype={9: str}
)

In [8]:
# Filter for one specific day
# Because of limited computational resources, I restrict the dataset to a single day, which gives me 96 observations.
#df = df[df['date'].str.startswith("2025-04-01")]
df["date"] = pd.to_datetime(df["date"])
df.head()

,date,station_uuid,diesel,e5,e10,uuid,name,brand,street,house_number,...,in_thuringia,in_region,last_seen,is_open,weekday,holiday,time_sin,time_cos,brand_cat,Brent_Price
0,2025-04-01 00:00:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,False,1,2025-04-30 21:45:00,True,1,0,0.000000,1.000000,74.0,74.959999
1,2025-04-01 00:15:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,False,1,2025-04-30 21:45:00,True,1,0,0.065403,0.997859,74.0,74.959999
2,2025-04-01 00:30:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,False,1,2025-04-30 21:45:00,True,1,0,0.130526,0.991445,74.0,74.959999
3,2025-04-01 00:45:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,False,1,2025-04-30 21:45:00,True,1,0,0.195090,0.980785,74.0,74.959999
4,2025-04-01 01:00:00,00060065-7890-4444-8888-acdc00000004,1.559,1.709,1.649,00060065-7890-4444-8888-acdc00000004,Georg Ultsch GmbH,Tankstelle Lichtenfels,Robert-Koch-Str.,18,...,False,1,2025-04-30 21:45:00,True,1,0,0.258819,0.965926,74.0,74.959999


# 2. STGNN Design

In [11]:
# =============================
# a) Parameters
# =============================

STATION_COL = "station_uuid"
LAT_COL = "latitude"
LON_COL = "longitude"
TIME_COL = "date"

# Default-Targets
TARGET_COLS_DEFAULT = ["e5", "e10"]
FEATURE_COLS_DEFAULT = ["e5", "e10"]  # standard: Targets = Features

RADIUS_KM = 10
NEIGHBOUR = 5

EMBED_DIM = 32
HIDDEN_DIM = 64
LR = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50
PATIENCE = 10
VAL_SPLIT = 0.15
TEST_SPLIT = 0.15
SEED = 42

WINDOW_SIZE = 16
HORIZON_STEPS = 4  # Horizont per Target


# =============================
# b) Helper functions
# =============================

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def km_to_radians(km: float) -> float:
    earth_radius_km = 6371.0088
    return km / earth_radius_km


def build_knn_graph(
    lat_lon_deg: np.ndarray,
    k: int = 5,
    return_dists: bool = False,
    RADIUS_KM: Optional[float] = None
) -> np.ndarray | Tuple[np.ndarray, np.ndarray]:
    
    lat_lon_rad = np.radians(lat_lon_deg)
    tree = BallTree(lat_lon_rad, metric="haversine")

    dist_rad, ind = tree.query(lat_lon_rad, k=k + 1)

    src, dst, dists_km = [], [], []
    radius_rad = km_to_radians(RADIUS_KM) if RADIUS_KM is not None else None

    for i, (neigh_idx, neigh_dist_rad) in enumerate(zip(ind, dist_rad)):
        for j, dij_rad in zip(neigh_idx[1:], neigh_dist_rad[1:]):
            if radius_rad is not None and dij_rad > radius_rad:
                continue

            dij_km = dij_rad * 6371.0088  # Conversion for output
            src.append(j)
            dst.append(i)
            dists_km.append(dij_km)

    edge_index = np.vstack([np.array(src, dtype=int), np.array(dst, dtype=int)])

    if return_dists:
        return edge_index, np.asarray(dists_km, dtype=float)
    return edge_index


# =============================
# c) Window-based ST data (multi-step, multi-target)
# =============================

@dataclass
class STWindowDataMulti:
    x: torch.Tensor             # [T, N, F] full time series, F = num_features
    y: torch.Tensor             # [T, N, H * num_targets] H Horizonte für alle Targets
    edge_index: torch.Tensor
    train_end_idx: np.ndarray   # time indices where windows end (train)
    val_end_idx: np.ndarray     # time indices (validation)
    test_end_idx: np.ndarray    # time indices (test)
    valid_nodes: np.ndarray     # node indices (z.B. Thüringen)
    times: np.ndarray           # sorted timestamps
    meta: pd.DataFrame          # station metadata (uuid, lat, lon, node_id)
    feature_cols: List[str]     # relevant Feature-Spalten
    target_cols: List[str]      # relevant Target-Spalten


def make_window_data_multi_from_df(
    df: pd.DataFrame,
    window_size: int = WINDOW_SIZE,
    horizon_steps: int = HORIZON_STEPS,
    feature_cols: Optional[List[str]] = None,
    target_cols: Optional[List[str]] = None,
    stride: int = 1, 
    WINDOW_SIZE: int = 16,
    HORIZON_STEPS: int = 4,
) -> STWindowDataMulti:

    if target_cols is None:
        target_cols = TARGET_COLS_DEFAULT
    if feature_cols is None:
        feature_cols = FEATURE_COLS_DEFAULT

    # Necessary Columns
    base_cols = [STATION_COL, LAT_COL, LON_COL, TIME_COL, "in_thuringia"]
    all_val_cols = sorted(set(feature_cols) | set(target_cols))
    needed = base_cols + all_val_cols
    assert all(c in df.columns for c in needed), f"Missing columns: {set(needed) - set(df.columns)}"

    df = df.copy()

    # Stations + node ids
    stations = df[[STATION_COL, LAT_COL, LON_COL]].drop_duplicates().reset_index(drop=True)
    stations["node_id"] = np.arange(len(stations))
    station2id = stations.set_index(STATION_COL)["node_id"].to_dict()

    # Time axis
    times = np.sort(df[TIME_COL].unique())
    time2id = {t: i for i, t in enumerate(times)}

    N = len(stations)
    T = len(times)
    M = len(target_cols)             # Number Targets
    F = len(feature_cols)           # Number Features
    H = horizon_steps

    # Spatial graph
    lat_lon = stations[[LAT_COL, LON_COL]].to_numpy(dtype=float)
    edge_index_np = build_knn_graph(
        lat_lon_deg=lat_lon,
        k=NEIGHBOUR,
        RADIUS_KM=RADIUS_KM
    )
    edge_index = torch.tensor(edge_index_np, dtype=torch.long)

    # Tensors
    x = torch.zeros((T, N, F), dtype=torch.float32)              # [T, N, F]
    y = torch.full((T, N, H * M), float("nan"), dtype=torch.float32)  # [T, N, H*M]

    # Aggregation per (station, time) across all relevant columns
    df_idx = (
        df[[STATION_COL, TIME_COL] + all_val_cols]
        .groupby([STATION_COL, TIME_COL], as_index=False)
        .mean()
        .sort_values(TIME_COL)
    )

    df_idx["node_id"] = df_idx[STATION_COL].map(station2id)
    df_idx["time_id"] = df_idx[TIME_COL].map(time2id)

    # x[t, n, f] = Features at time t
    for _, row in df_idx.iterrows():
        t = int(row["time_id"])
        n = int(row["node_id"])
        for f_idx, col in enumerate(feature_cols):
            v = row[col]
            if pd.notna(v):
                x[t, n, f_idx] = float(v)

    # y[t, n, m*H + h] = Target m at time t+h+1
    for station, g in df_idx.groupby(STATION_COL):
        g = g.sort_values("time_id")
        node_id = int(g["node_id"].iloc[0])
        t_ids = g["time_id"].to_numpy()

        L = len(t_ids)
        for m, col in enumerate(target_cols):
            vals = g[col].to_numpy()

            for idx in range(L):
                t = int(t_ids[idx])
                if idx + H >= L:
                    break

                future_vals = vals[idx + 1: idx + 1 + H]

                # If there are NaNs in the horizon for this target, skip this start time
                if np.any(np.isnan(future_vals)):
                    continue

                for h in range(H):
                    y[t, node_id, m * H + h] = float(future_vals[h])

    # Time indices where full horizon is available
    effective_T = T - H
    if effective_T < window_size:
        raise ValueError(f"Too few timesteps ({effective_T}) for window size {window_size} and horizon {H}.")

    # Time-based split over 0..effective_T-1
    time_indices = np.arange(effective_T)
    num_total = len(time_indices)
    num_train = int((1.0 - VAL_SPLIT - TEST_SPLIT) * num_total)
    num_val   = int(VAL_SPLIT * num_total)
    num_test  = num_total - num_train - num_val

    train_t = time_indices[:num_train]
    val_t   = time_indices[num_train:num_train + num_val]
    test_t  = time_indices[num_train + num_val:]

    # Window end indices
    min_end = window_size - 1
    train_end_idx = train_t[train_t >= min_end]
    val_end_idx   = val_t[val_t >= min_end]
    test_end_idx  = test_t[test_t >= min_end]
    
    #30min
    train_end_idx = train_end_idx[::stride]
    val_end_idx   = val_end_idx[::stride]
    test_end_idx  = test_end_idx[::stride]


    # Thuringia mask
    th_mask = (
        df[[STATION_COL, "in_thuringia"]]
        .drop_duplicates()
        .set_index(STATION_COL)["in_thuringia"]
    )
    stations = stations.join(th_mask, on=STATION_COL)
    valid_nodes = stations.index[stations["in_thuringia"] == 1].to_numpy()

    if valid_nodes.size == 0:
        raise ValueError("No nodes with in_thuringia == 1 found.")

    meta = stations[[STATION_COL, LAT_COL, LON_COL, "node_id"]].copy()

    return STWindowDataMulti(
        x=x,
        y=y,
        edge_index=edge_index,
        train_end_idx=train_end_idx,
        val_end_idx=val_end_idx,
        test_end_idx=test_end_idx,
        valid_nodes=valid_nodes,
        times=times,
        meta=meta,
        feature_cols=feature_cols,
        target_cols=target_cols,
    )


# =============================
# d) STGNN with windowing (multi-step, multi-target)
# =============================

import torch
import torch.nn as nn
from torch_geometric.nn import SAGEConv, GatedGraphConv, GATConv


class STGNNWindowMultiRegressor(nn.Module):

    def __init__(
        self,
        num_nodes: int,
        embed_dim: int,
        hidden_dim: int,
        out_dim: int,
        in_dim: int = 1,
        gnn_type: str = "sage",           # "sage", "ggnn", "gat", ...
        temporal_type: str = "gru",       # "gru", "lstm", "attn"
        attn_heads: int = 4,              # only for "attn"
        spatial_pool: str = "mean",       # "mean" oder "last"
    ):
        super().__init__()
        self.out_dim = out_dim
        self.gnn_type = gnn_type
        self.temporal_type = temporal_type
        self.hidden_dim = hidden_dim
        self.spatial_pool = spatial_pool

        # ----- Node-Embedding + Input-Projektion -----
        self.emb = nn.Embedding(num_nodes, embed_dim)
        self.proj = nn.Linear(in_dim + embed_dim, hidden_dim)
        self.act = nn.ReLU()

        # ----- spatial GNN -----
        if gnn_type == "sage":
            self.gnn_layer = SAGEConv(hidden_dim, hidden_dim)
        elif gnn_type == "ggnn":
            self.gnn_layer = GatedGraphConv(out_channels=hidden_dim, num_layers=1)
        elif gnn_type == "gat":
            self.gnn_layer = GATConv(
                in_channels=hidden_dim,
                out_channels=hidden_dim,
                heads=1,
                concat=False,
            )
        else:
            raise ValueError(f"Unknown gnn_type: {gnn_type}")

        # ----- temporary module -----
        if temporal_type == "gru":
            self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=False)
        elif temporal_type == "lstm":
            self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=False)
        elif temporal_type == "attn":
            self.attn = nn.MultiheadAttention(
                embed_dim=hidden_dim,
                num_heads=attn_heads,
                batch_first=False,  # [W, N, H]
            )
        else:
            raise ValueError(f"Unknown temporal_type: {temporal_type}")

        # ----- Gating fusion + head -----
        # Gate receives [h_spat || h_temp] and outputs a feature gate [N, H]
        self.gate_layer = nn.Linear(2 * hidden_dim, hidden_dim)
        self.head = nn.Linear(hidden_dim, out_dim)

    # -------------------------------------------------
    # Auxilary functions
    # -------------------------------------------------
    def _apply_spatial(self, h: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        """
        h: [N, hidden_dim]
        """
        h = self.gnn_layer(h, edge_index)
        h = self.act(h)
        return h

    def _apply_temporal(self, seq: torch.Tensor) -> torch.Tensor:
        """
        seq: [W, N, hidden_dim]
        Rückgabe: [N, hidden_dim]
        """
        if self.temporal_type == "gru":
            out, h_n = self.rnn(seq)         # h_n: [num_layers, N, H]
            last = h_n[-1]                   # [N, H]
            return last

        elif self.temporal_type == "lstm":
            out, (h_n, c_n) = self.rnn(seq)  # h_n: [num_layers, N, H]
            last = h_n[-1]                   # [N, H]
            return last

        elif self.temporal_type == "attn":
            attn_out, _ = self.attn(seq, seq, seq)  # [W, N, H]
            last = attn_out[-1]                     # [N, H]
            # last = attn_out.mean(dim=0)
            return last

        else:
            raise ValueError(f"Unknown temporal_type: {self.temporal_type}")

    # -------------------------------------------------
    # Parallel-Forward with Gating
    # -------------------------------------------------
    def forward(self, x_window: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        """
        x_window: [W, N, F]
        edge_index: [2, E]
        """
        W, N, F = x_window.shape

        # shared hidden h_seq: [W, N, H]
        node_emb = self.emb.weight                                # [N, embed_dim]
        node_emb_exp = node_emb.unsqueeze(0).expand(W, -1, -1)    # [W, N, embed_dim]

        h = torch.cat([x_window, node_emb_exp], dim=2)            # [W, N, F + embed_dim]
        h = self.proj(h)                                          # [W, N, H]
        h = self.act(h)

        # ----- temporal branch -----
        h_temp = self._apply_temporal(h)                          # [N, H]

        # ----- spatial branch -----
        gnn_out_per_t = []
        for t in range(W):
            ht = h[t]                                             # [N, H]
            ht_spat = self._apply_spatial(ht, edge_index)         # [N, H]
            gnn_out_per_t.append(ht_spat)

        h_spat_seq = torch.stack(gnn_out_per_t, dim=0)            # [W, N, H]

        if self.spatial_pool == "mean":
            h_spat = h_spat_seq.mean(dim=0)                       # [N, H]
        elif self.spatial_pool == "last":
            h_spat = h_spat_seq[-1]                               # [N, H]
        else:
            raise ValueError(f"Unknown spatial_pool: {self.spatial_pool}")

        # ----- Gating-Fusion -----
        # concat: [N, 2H]
        h_cat = torch.cat([h_spat, h_temp], dim=-1)

        # Gate: [N, H] with values in (0, 1)
        gate = torch.sigmoid(self.gate_layer(h_cat))

        # Feature-wise blending of spatial and temporal
        h_fused = gate * h_spat + (1.0 - gate) * h_temp           # [N, H]
        h_fused = self.act(h_fused)

        out = self.head(h_fused)                                  # [N, out_dim]
        return out

    
# =============================
# e) Training and evaluation (multi-step, multi-target)
# =============================

@dataclass
class STTrainConfig:
    lr: float = LR
    weight_decay: float = WEIGHT_DECAY
    epochs: int = EPOCHS
    patience: int = PATIENCE
    window_size: int = WINDOW_SIZE
    horizon_steps: int = HORIZON_STEPS  # per Target


def _compute_window_metrics_multi(
    model: nn.Module,
    data: STWindowDataMulti,
    end_indices: np.ndarray,
    device: torch.device,
    loss_nodes: torch.Tensor,
    window_size: int,   
) -> dict:

    x_full = data.x.to(device)
    y_full = data.y.to(device)
    edge_index = data.edge_index.to(device)
    W = window_size

    preds_all = []
    trues_all = []

    model.eval()
    with torch.no_grad():
        for end_t in end_indices:
            start_t = end_t - W + 1
            x_win = x_full[start_t:end_t+1]
            y_target = y_full[end_t]     # [N, H * M]

            pred = model(x_win, edge_index)  # [N, H * M]

            y_nodes = y_target[loss_nodes]
            p_nodes = pred[loss_nodes]

            mask = torch.isfinite(y_nodes)
            if mask.sum() == 0:
                continue

            preds_all.append(p_nodes[mask].cpu().numpy())
            trues_all.append(y_nodes[mask].cpu().numpy())

    if len(preds_all) == 0:
        return {"MAE": float("nan"), "RMSE": float("nan")}

    preds_concat = np.concatenate(preds_all)
    trues_concat = np.concatenate(trues_all)

    mae = np.mean(np.abs(preds_concat - trues_concat))
    mse = np.mean((preds_concat - trues_concat) ** 2)
    rmse = math.sqrt(mse)

    return {"MAE": float(mae), "RMSE": float(rmse)}


def train_stgnn_window_multi(
    data: STWindowDataMulti,
    cfg: STTrainConfig,
    gnn_type: str = "sage",
    temporal_type: str = "gru",
    attn_heads: int = 4,
) -> dict:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_full = data.x.to(device)
    y_full = data.y.to(device)
    edge_index = data.edge_index.to(device)
    W = cfg.window_size

    M = len(data.target_cols)
    H_per_target = cfg.horizon_steps
    out_dim = H_per_target * M   # H * num_targets
    in_dim = x_full.shape[2]     # num_features

    model = STGNNWindowMultiRegressor(
        num_nodes=x_full.shape[1],
        embed_dim=EMBED_DIM,
        hidden_dim=HIDDEN_DIM,
        out_dim=out_dim,
        in_dim=in_dim,
        gnn_type=gnn_type,
        temporal_type=temporal_type,
        attn_heads=attn_heads,
    ).to(device)

    opt = AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    loss_fn = nn.SmoothL1Loss()

    loss_nodes = torch.tensor(data.valid_nodes, dtype=torch.long, device=device)

    best_state = None
    best_val = float("inf")
    wait = 0

    for epoch in range(cfg.epochs):
        model.train()
        epoch_loss = 0.0
        count = 0

        for end_t in data.train_end_idx:
            start_t = end_t - W + 1

            x_win = x_full[start_t:end_t+1]
            y_target = y_full[end_t]   # [N, H*M]

            pred = model(x_win, edge_index)  # [N, H*M]

            y_nodes = y_target[loss_nodes]
            p_nodes = pred[loss_nodes]

            mask = torch.isfinite(y_nodes)
            if mask.sum() == 0:
                continue

            loss = loss_fn(p_nodes[mask], y_nodes[mask])

            opt.zero_grad()
            loss.backward()
            opt.step()

            epoch_loss += loss.item()
            count += 1

        avg_train_loss = epoch_loss / max(count, 1)

        val_metrics = _compute_window_metrics_multi(model, data, data.val_end_idx, device, loss_nodes, cfg.window_size)
        val_mae = val_metrics["MAE"]

        #print(f"Epoch {epoch+1}: train_loss={avg_train_loss:.4f}, val_MAE={val_mae:.4f}")

        if val_mae < best_val:
            best_val = val_mae
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= cfg.patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    test_metrics = _compute_window_metrics_multi(model, data, data.test_end_idx, device, loss_nodes, cfg.window_size)
    val_metrics = _compute_window_metrics_multi(model, data, data.val_end_idx, device, loss_nodes, cfg.window_size)

    return {
        "model": model,
        "metrics": {
            "val": val_metrics,
            "test": test_metrics,
        },
    }


# =============================
# f) High-level API
# =============================

def train_stgnn_window_from_df(
    df: pd.DataFrame,
    *,
    seed: int = SEED,
    window_size: int = WINDOW_SIZE,
    horizon_steps: int = HORIZON_STEPS,
    feature_cols: Optional[List[str]] = None,
    target_cols: Optional[List[str]] = None,
    stride: int = 1, 
    gnn_type: str = "sage",   
    temporal_type: str = "gru",
    attn_heads: int = 4,
) -> dict:

    set_seed(seed)
    data = make_window_data_multi_from_df(
        df,
        window_size=window_size,
        horizon_steps=horizon_steps,
        feature_cols=feature_cols,
        target_cols=target_cols,
        stride=stride, 
    )
    cfg = STTrainConfig(window_size=window_size, horizon_steps=horizon_steps)
    result = train_stgnn_window_multi(
        data,
        cfg,
        gnn_type=gnn_type,
        temporal_type=temporal_type,
        attn_heads=attn_heads,
    )

    return {
        "model": result["model"],
        "metrics": result["metrics"],
        "data": data,
        "window_size": window_size,
        "horizon_steps": horizon_steps,
        "feature_cols": data.feature_cols,
        "target_cols": data.target_cols,
    }

In [16]:
# Baseline Experiment
Baseline = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size=8,
    horizon_steps=2,    
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Baseline")
print("Val:", Baseline["metrics"]["val"])
print("Test:", Baseline["metrics"]["test"])

Baseline
Val: {'MAE': 0.015028496272861958, 'RMSE': 0.019202405499193314}
Test: {'MAE': 0.025361239910125732, 'RMSE': 0.033157957785040784}


# 3. Experiments

## 3.1 Testing different Targets

In [ ]:
# Experiment 1a - E10
Experiment1a = train_stgnn_window_from_df(
    df,
    feature_cols=["e10"],
    target_cols=["e10"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 1a - E10")
print("Val:", Experiment1a["metrics"]["val"])
print("Test:", Experiment1a["metrics"]["test"])

In [ ]:
# Experiment 1b - Diesel
Experiment1b = train_stgnn_window_from_df(
    df,
    feature_cols=["diesel"],
    target_cols=["diesel"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 1b - Diesel")
print("Val:", Experiment1b["metrics"]["val"])
print("Test:", Experiment1b["metrics"]["test"])

In [ ]:
# Experiment 1c - Alle petrol types
Experiment1c = train_stgnn_window_from_df(
    df,
    feature_cols=["e5", "e10", "diesel"],
    target_cols=["e5", "e10", "diesel"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 1c - All Petrol Types")
print("Val:", Experiment1c["metrics"]["val"])
print("Test:", Experiment1c["metrics"]["test"])

## 3.2. Testing different Features

In [ ]:
# Experiment 2a - Only Prices 
Experiment2a = train_stgnn_window_from_df(
    df,
    feature_cols=["e5", "e10", "diesel"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)
print("Experiment 2a - Only Prices")
print("Val:", Experiment2a["metrics"]["val"])
print("Test:", Experiment2a["metrics"]["test"])

In [ ]:
# Experiment 2b - Prices and Days
Experiment2b = train_stgnn_window_from_df(
    df,
    feature_cols=["e5", "e10", "diesel", "weekday", "holiday"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 2b - Prices and Days")
print("Val:", Experiment2b["metrics"]["val"])
print("Test:", Experiment2b["metrics"]["test"])

In [ ]:
# Experiment 2c - Prices, Days and Time
Experiment2c = train_stgnn_window_from_df(
    df,
    feature_cols=["e5", "e10", "diesel", "weekday", "holiday", "time_sin"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 2c - Prices, Days and Time")
print("Val:", Experiment2c["metrics"]["val"])
print("Test:", Experiment2c["metrics"]["test"])

In [ ]:
# Experiment 2d - Prices, Days, Time and Brand
Experiment2d = train_stgnn_window_from_df(
    df,
    feature_cols=["e5", "e10", "diesel", "weekday", "holiday", "time_sin", "brand_cat"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 2d - Prices, Days, Time and Brand")
print("Val:", Experiment2d["metrics"]["val"])
print("Test:", Experiment2d["metrics"]["test"])

In [ ]:
# Experiment 2e - Full features
Experiment2e = train_stgnn_window_from_df(
    df,
    feature_cols=["e5", "e10", "diesel", "weekday", "holiday", "time_sin", "brand_cat", "Brent_Price"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 2e - Full features")
print("Val:", Experiment2e["metrics"]["val"])
print("Test:", Experiment2e["metrics"]["test"])

## 3.3. Testing different Window Sizes

In [ ]:
# Experiment 3a - Window Size: 16
Experiment3a = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 16,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 3a - Window Size: 16")
print("Val:", Experiment3a["metrics"]["val"])
print("Test:", Experiment3a["metrics"]["test"])

In [ ]:
# Experiment 3b - Window Size: 32
Experiment3b = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 32,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 3b - Window Size: 32")
print("Val:", Experiment3b["metrics"]["val"])
print("Test:", Experiment3b["metrics"]["test"])

In [ ]:
# Experiment 3c - Window Size: 48
Experiment3c = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 48,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 3c - Window Size: 48")
print("Val:", Experiment3c["metrics"]["val"])
print("Test:", Experiment3c["metrics"]["test"])

In [ ]:
# Experiment 3d - Window Size: 96
Experiment3d = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 96,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 3d - Window Size: 96")
print("Val:", Experiment3d["metrics"]["val"])
print("Test:", Experiment3d["metrics"]["test"])

## 3.4. Testing different Horizons

In [ ]:
# Experiment 4a - Horizon: 4
Experiment4a = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 4,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 4a - Horizon: 4")
print("Val:", Experiment4a["metrics"]["val"])
print("Test:", Experiment4a["metrics"]["test"])

In [12]:
# Experiment 4b - Horizon: 8
Experiment4b = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size=8,
    horizon_steps=8,   
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 4b - Horizon: 8")
print("Val:", Experiment4b["metrics"]["val"])
print("Test:", Experiment4b["metrics"]["test"])

Experiment 4b - Horizon: 8
Val: {'MAE': 0.015349118039011955, 'RMSE': 0.019514059906244698}
Test: {'MAE': 0.02540482021868229, 'RMSE': 0.0339113739422546}


## 3.5 Testing different Time Intervalls

In [ ]:
# Experiment 5a - Time Intervall: 30min
Experiment5a = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 2,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 5a - Time Intervall: 30min")
print("Val:", Experiment5a["metrics"]["val"])
print("Test:", Experiment5a["metrics"]["test"])

In [ ]:
# Experiment 5b - Time Intervall: 60min
Experiment5b = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 4,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 5b - Time Intervall: 60min")
print("Val:", Experiment5b["metrics"]["val"])
print("Test:", Experiment5b["metrics"]["test"])

## 3.7 Testing different GNN Types

In [ ]:
# Experiment 7a - Recurrent GNN
Experiment7a = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "ggnn",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 7a - Recurrent GNN")
print("Val:", Experiment7a["metrics"]["val"])
print("Test:", Experiment7a["metrics"]["test"])

In [ ]:
# Experiment 7b - Attentional GNN
Experiment7b = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "gat",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "gru",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 7b - Attentional GNN")
print("Val:", Experiment7b["metrics"]["val"])
print("Test:", Experiment7b["metrics"]["test"])

## 3.8 Testing different Temporal Modules

In [ ]:
# Experiment 8a - LSTM
Experiment8a = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "lstm",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 8a - LSTM")
print("Val:", Experiment8a["metrics"]["val"])
print("Test:", Experiment8a["metrics"]["test"])

In [ ]:
# Experiment 8b - Self-Attention
Experiment8b = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "sage",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "attn",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 8b - Self-Attention")
print("Val:", Experiment8b["metrics"]["val"])
print("Test:", Experiment8b["metrics"]["test"])

In [ ]:
# Experiment 8c - Self-Attention and Attentional GNN
Experiment8c = train_stgnn_window_from_df(
    df,
    feature_cols=["e5"],
    target_cols=["e5"],
    window_size = 8,
    horizon_steps = 2,
    stride= 1,             # Intveralls: 1--> 15min // 2 --> 30min // 4 --> 60min,
    gnn_type= "gat",      # Convolutional GNN: sage // Recurrent GNN: ggnn // Attentional GNN: gat,
    temporal_type= "attn",  # GRU: gru // LSTM: lstm // Self-Attention: attn
)

print("Experiment 8c - Self-Attention")
print("Val:", Experiment8c["metrics"]["val"])
print("Test:", Experiment8c["metrics"]["test"])